In [4]:
import os
import pickle
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
path = os.path.join(os.getcwd(),r'..\data\Rating and Reviews')
os.listdir(path)

['cosmetic.p',
 'Cosmetics_Reviews_20190630.csv',
 'Cosmetics_Reviews_20190831.csv',
 'Cosmetics_Reviews_20190930.csv',
 'Cosmetics_Reviews_20191031.csv',
 'cosmetics_reviews_20191130.csv',
 'cosmetics_reviews_20191130_final.csv',
 'cosmetics_reviews_20191231.csv',
 'cosmetics_reviews_20200101-20200131.csv',
 'cosmetics_reviews_20200229.csv',
 'pwds',
 'Skin Care_Reviews_20190930.csv',
 'skincare_reviews_20150201-20200131.csv',
 'SkinCare_Reviews_20190630_ST MV2SV.csv',
 'SkinCare_Reviews_20190831.csv',
 'SkinCare_Reviews_20190930.7z',
 'SkinCare_Reviews_20191031.csv',
 'skincare_reviews_20191130.csv',
 'skincare_reviews_20191231.csv',
 'skincare_reviews_20200229.csv']

## Reading and concatenating all datasets together (do not run)
Bad idea, since some of the files are bigger than 5 Gb.

In [9]:
# skincare = pd.DataFrame()
# cosmetic = pd.DataFrame()
# for file in listdir(path):
#     if '.csv' in file.lower():
#         if 'skincare' in file.lower():
#             print('Adding', file)
#             skincare=pd.concat([skincare, pd.read_csv(os.path.join(path,file))])
#             display(skincare)
#         elif 'cosmetics' in file.lower():
#             print('Adding', file)
#             cosmetic=pd.concat([cosmetic, pd.read_csv(os.path.join(path,file))])
#             display(cosmetic)

In [ ]:
# skincare = dict()
# cosmetic = dict()
# for file in listdir(path):
#     if '.csv' in file.lower():
#         if 'skincare' in file.lower():
#             print('Adding', file)
#             skincare[file] = pd.read_csv(os.path.join(path,file))
# #         elif 'cosmetics' in file.lower():
# #             print('Adding', file)
# #             cosmetic[file] = pd.read_csv(os.path.join(path,file))

In [ ]:
# pickle.dump(cosmetic, open( "../data/Rating and Reviews/cosmetic.p", "wb" ) )

In [7]:
# cosmetic_df=pd.DataFrame()
# for key, df in cosmetic.items():
#     print(key)
#     print(df.columns)
#     example=df
#     break

## Data wrangling and aggregation
1. Filter only on US
2. create data by YM
3. change sentiment to num_sentiment : -1,0,1
4. change sentiments to hot ones HO_sentiment
5. create hot ones rating
5. Groupby YM, Source Product Identifierm, Channel, Product_ID
6. Aggregate by {num_sentiment:mean, num_rating:mean, HO_sentiments:count, HO_rating:count}
7. Number of reviews

In [8]:
df = pd.read_csv('../data/Rating and Reviews/Cosmetics_Reviews_20191031.csv')

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
example[example['Geography']=='USA']['Product_ID'].isna().sum()/len(example[example['Geography']=='USA'])